In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [ ]:
# Splitter datasettet i trenings- og test-data på nytt, siden vi har forandret på deliveries
from sklearn.model_selection import train_test_split
X, y = deliveries.drop("service_time", axis=1), deliveries["service_time"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)